#### Linear Regeression : In Depth

In [ ]:
# We begin with the standard imports
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
sns.set()


In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + rng.randn(50)
plt.scatter(x, y)


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True)

model.fit(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)


In [ ]:
print(f"model slope = {model.coef_[0]}\nmodel intercept = {model.intercept_}")

The LinearRegression estimator can also handle higher dimensional models but the\
complexity in visualizing the model also increases as well

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
y = 0.5 + np.dot(X, [1.5, -2.0, 1.0])

model.fit(X, y)
print(model.coef_)
print(model.intercept_)


##### Polynomial basis functions

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

x = np.array(
    [
        2,
        3,
        4,
    ]
)
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

we can see that the transformer has converted our 1D array into a 3D array by taking the exponent of each value.\
We can then plug the new higher dimensional model into a linear regression

In [ ]:
from sklearn.pipeline import make_pipeline

poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())


In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

poly_model.fit(x[:, np.newaxis], y)
xfit = np.linspace(0, 10, 1000)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)

##### Gaussian basis functions
Since they are not built into sklearn, let's create our own

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


class GaussianFeatures(BaseEstimator, TransformerMixin):

    """Uniformly spaced Gaussian features for one-dimensional input"""

    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor

    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg**2, axis))

    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self

    def transform(self, X):
        return self._gauss_basis(
            X[:, :, np.newaxis], self.centers_, self.width_, axis=1
        )

In [ ]:
gauss_model = make_pipeline(GaussianFeatures(20), LinearRegression())

gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)
plt.xlim(0, 15)


#### Regularization
The use of basis functions can quickly lead to overfitting e.g

In [ ]:
model = make_pipeline(GaussianFeatures(30), LinearRegression())

model.fit(x[:, np.newaxis], y)

plt.scatter(x, y)
plt.plot(xfit, model.predict(xfit[:, np.newaxis]))
plt.xlim(0, 10)
plt.ylim(-1.5, 1.5)


We can see that the model becomes too much flexible and even goes to extreme values between locations where it is constrained by data.\
We can see the reason for this if we plot the coefficients of the Gaussian bases in respect to their locations

In [ ]:
def basis_plot(model, title=None):
    fig, ax = plt.subplots(2, sharex=True)
    model.fit(x[:, np.newaxis], y)
    ax[0].scatter(x, y)
    ax[0].plot(xfit, model.predict(xfit[:, np.newaxis]))
    ax[0].set(xlabel="x", ylabel="y", ylim=(-1.5, 1.5))

    if title:
        ax[0].set_title(title)

    ax[1].plot(model.steps[0][1].centers_, model.steps[1][1].coef_)
    ax[1].set(xlabel="basis location", ylabel="coefficient", xlim=(0, 10))

In [ ]:
model = make_pipeline(GaussianFeatures(30), LinearRegression())

basis_plot(model)

##### 1. Ridge regression (Tikhonov regularization)

In [ ]:
from sklearn.linear_model import Ridge

model =  make_pipeline(GaussianFeatures(30), Ridge(alpha=0.1))

basis_plot(model, title='Ridge regression')

##### 2.Lasso regularization

In [ ]:
from sklearn.linear_model import Lasso

model =  make_pipeline(GaussianFeatures(30), Lasso(alpha=0.001))

basis_plot(model, title='Ridge regression')

In both(and generally all) types of regularization, the knob 'alpha' which decides the strength of the penalty\
should be determined by validation techniques e.g Cross-Validation 